In [1]:
import torch
import argparse
from model import MODEL
from run import train, test
import numpy as np
import torch.optim as optim
from dataloader import getDataLoader
from earlystop import EarlyStopping

In [11]:
gpu = 0
patience = 5
max_iter = 100  # number of iterations
init_lr = 0.005  # initial learning rate
lr_decay = 0.75  # learning rate decay
final_lr = 1E-5  # learning rate will not decrease after hitting this threshold
dropout = 0.3
exercise_embed_dim = 64  # hidden state dim for LSTM
layer_num = 2  # layer number for LSTM
max_step = 100  # the allowed maximum length of a sequence
maxgradnorm = 50.0  # maximum gradient norm
Lamda = 1  # hyper-parameter Lamda
alpha = 0.2  # alpha
num_heads = 4  # number of head attentions
mode = 3  # mode of integration function
# 1: ca / 2: mul / 3: ca mul / 4: rasch
fold = '1'  # number of fold
dataset = 'assist2009_B'

#when dataset is assist2009_B
batch_size = 32
n_knowledge_concept = 110  # the kinds of unique questions in the dataset
n_exercise = 16891  # the number of unique questions in the dataset
data_dir = './data/assist2009_B'  # data directory
data_name = 'assist2009_B'  # data set name

memory_size = n_knowledge_concept
lr = init_lr
memory_key_state_dim = exercise_embed_dim
memory_value_state_dim = exercise_embed_dim * 2

train_data_path = data_dir + "/" + data_name + "_train" + fold + ".csv"
valid_data_path = data_dir + "/" + data_name + "_valid" + fold + ".csv"
test_data_path = data_dir + "/" + data_name + "_test" + fold + ".csv"

In [52]:
import numpy as np
import itertools
class DataReader():
    def __init__(self, train_path, valid_path, test_path, maxstep, numofques):
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.maxstep = maxstep
        self.numofques = numofques

    def getTrainData(self):
        print('loading train data...')
        q_data= []
        qa_data =[]
        p_data= []
        trainData = []
        batch = 0
        with open(self.train_path, 'r') as train:
            for student, problem, ques, ans in itertools.zip_longest(*[train] * 4):
                batch = batch +1
                try:
                    problem = [int(p) for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length//self.maxstep + (1 if length%self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i*self.maxstep + j]
                            qa_temp[j] = ans[i*self.maxstep + j]
                            p_temp[j] = problem[i*self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('train--question done: ' + str(np.array(q_data).shape))
            print('train--question_ans done: ' + str(np.array(qa_data).shape))
            print('train--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)

    def getValidData(self):
        print('loading valid data...')
        q_data= []
        qa_data =[]
        p_data= []
        validData = []
        batch = 0
        with open(self.valid_path, 'r') as valid:
            for student, problem, ques, ans in itertools.zip_longest(*[valid] * 4):
                batch = batch +1
                try:
                    problem = [int(p) for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length//self.maxstep + (1 if length%self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i*self.maxstep + j]
                            qa_temp[j] = ans[i*self.maxstep + j]
                            p_temp[j] = problem[i*self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('valid--question done: ' + str(np.array(q_data).shape))
            print('valid--question_ans done: ' + str(np.array(qa_data).shape))
            print('valid--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)


    def getTestData(self):
        print('loading test data...')
        testData = []
        q_data = []
        qa_data = []
        p_data = []
        zero = [0 for i in range(self.numofques * 2)]
        batch = 0
        with open(self.test_path, 'r') as test:
            for student, problem, ques, ans in itertools.zip_longest(*[test] * 4):
                #length = int(length.strip().strip(','))
                batch = batch + 1
                try:
                    problem = [int(p) for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length // self.maxstep + (1 if length % self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i * self.maxstep + j]
                            qa_temp[j] = ans[i * self.maxstep + j]
                            p_temp[j] = problem[i * self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('test--question done: ' + str(np.array(q_data).shape))
            print('test--question_ans done: ' + str(np.array(qa_data).shape))
            print('test--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)



In [4]:
import numpy as np
import itertools


class DataReader():
    def __init__(self, train_path, valid_path, test_path, maxstep, numofques):
        self.train_path = train_path
        self.valid_path = valid_path
        self.test_path = test_path
        self.maxstep = maxstep
        self.numofques = numofques

    def getTrainData(self):
        print('loading train data...')
        q_data = []
        qa_data = []
        p_data = []
        trainData = []
        batch = 0
        with open(self.train_path, 'r') as train:
            for student, problem, ques, ans in itertools.zip_longest(*[train] * 4):
                batch = batch + 1
                try:
                    problem = [int(p)
                               for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length//self.maxstep + \
                    (1 if length % self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i*self.maxstep + j]
                            qa_temp[j] = ans[i*self.maxstep + j]
                            p_temp[j] = problem[i*self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('train--question done: ' + str(np.array(q_data).shape))
            print('train--question_ans done: ' + str(np.array(qa_data).shape))
            print('train--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)

    def getValidData(self):
        print('loading valid data...')
        q_data = []
        qa_data = []
        p_data = []
        validData = []
        batch = 0
        with open(self.valid_path, 'r') as valid:
            for student, problem, ques, ans in itertools.zip_longest(*[valid] * 4):
                batch = batch + 1
                try:
                    problem = [int(p)
                               for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length//self.maxstep + \
                    (1 if length % self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i*self.maxstep + j]
                            qa_temp[j] = ans[i*self.maxstep + j]
                            p_temp[j] = problem[i*self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('valid--question done: ' + str(np.array(q_data).shape))
            print('valid--question_ans done: ' + str(np.array(qa_data).shape))
            print('valid--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)

    def getTestData(self):
        print('loading test data...')
        testData = []
        q_data = []
        qa_data = []
        p_data = []
        zero = [0 for i in range(self.numofques * 2)]
        batch = 0
        with open(self.test_path, 'r') as test:
            for student, problem, ques, ans in itertools.zip_longest(*[test] * 4):
                #length = int(length.strip().strip(','))
                batch = batch + 1
                try:
                    problem = [int(p)
                               for p in problem.strip().strip(',').split(',')]
                    ques = [int(q) for q in ques.strip().strip(',').split(',')]
                    ans = [int(a) for a in ans.strip().strip(',').split(',')]
                except:
                    print(batch)
                length = np.size(ques)
                slices = length // self.maxstep + \
                    (1 if length % self.maxstep > 0 else 0)
                for i in range(slices):
                    q_temp = np.zeros(shape=[self.maxstep])
                    qa_temp = np.zeros(shape=[self.maxstep])
                    p_temp = np.zeros(shape=[self.maxstep])
                    if length > 0:
                        if length >= self.maxstep:
                            l = self.maxstep
                        else:
                            l = length
                        for j in range(l):
                            q_temp[j] = ques[i * self.maxstep + j]
                            qa_temp[j] = ans[i * self.maxstep + j]
                            p_temp[j] = problem[i * self.maxstep + j]
                        length = length - self.maxstep
                    q_data.append(q_temp.tolist())
                    qa_data.append(qa_temp.tolist())
                    p_data.append(p_temp.tolist())
            print('test--question done: ' + str(np.array(q_data).shape))
            print('test--question_ans done: ' + str(np.array(qa_data).shape))
            print('test--problem done: ' + str(np.array(p_data).shape))
            return np.array(q_data).astype(float), np.array(qa_data).astype(float), np.array(p_data).astype(float)


In [53]:
import sys
sys.path.append('/')
from readdata import DataReader
import numpy as np
from tqdm import tqdm


def getDataLoader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept):
    handle = DataReader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept)

    kc_data_train, respose_data_train, exercise_data_train = handle.getTrainData()
    kc_data_valid, respose_data_valid, exercise_data_valid = handle.getValidData()
    kc_data_test, respose_data_test, exercise_data_test = handle.getTestData()

    return kc_data_train, respose_data_train, exercise_data_train,kc_data_valid, respose_data_valid, exercise_data_valid, kc_data_test, respose_data_test, exercise_data_test



In [68]:
train_kc_data, train_respond_data, train_exercise_data, \
        valid_kc_data, valid_respose_data, valid_exercise_data, \
        test_kc_data, test_respose_data, test_exercise_data \
        = getDataLoader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept)

loading train data...
train--question done: (3293, 100)
train--question_ans done: (3293, 100)
train--problem done: (3293, 100)
loading valid data...
valid--question done: (1038, 100)
valid--question_ans done: (1038, 100)
valid--problem done: (1038, 100)
loading test data...
test--question done: (1032, 100)
test--question_ans done: (1032, 100)
test--problem done: (1032, 100)


In [69]:
train_exercise_respond_data = train_respond_data * \
    n_exercise + train_exercise_data
valid_exercise_respose_data = valid_respose_data * \
    n_exercise + valid_exercise_data
test_exercise_respose_data = test_respose_data * \
    n_exercise + test_exercise_data
adj_exercise_kc = np.loadtxt(data_dir + "/exercise_kc_map.txt")


In [80]:
len(adj_exercise_kc)

16891

In [81]:
n_exercise

16891

In [6]:
memory_size = n_knowledge_concept
lr = init_lr
memory_key_state_dim = exercise_embed_dim
memory_value_state_dim = exercise_embed_dim * 2

train_data_path = data_dir + "/" + data_name + "_train" + fold + ".csv"
valid_data_path = data_dir + "/" + data_name + "_valid" + fold + ".csv"
test_data_path = data_dir + "/" + data_name + "_test" + fold + ".csv"

train_kc_data, train_respond_data, train_exercise_data, \
    valid_kc_data, valid_respose_data, valid_exercise_data, \
    test_kc_data, test_respose_data, test_exercise_data \
    = getDataLoader(train_data_path, valid_data_path, test_data_path, max_step, n_knowledge_concept)


TypeError: can only concatenate str (not "int") to str